In [21]:
import py_midicsv as pm
import csv
import pandas as pd
import math
import matplotlib.pyplot as plt

# Creating the Dataframe

In [ ]:
# https://www.fourmilab.ch/webtools/midicsv/
def extract_instruments_from_csv(file_path):
    instrument_names = []

    # Open the CSV file
    with open(file_path, mode='r') as file:
        reader = csv.reader(file)
        
        # Traverse the CSV file line by line
        for row in reader:
            if " Text_t" in row:
                # Extract the instrument name that follows 'Text_t'
                # print(row)
                record_number = row[0].strip()
                # print("record_number: ", record_number)
                instrument_name = row[3].strip().strip('"')
                instrument_names.append([record_number, instrument_name])
                # print(instrument_names)
    return instrument_names

def calculate_note_durations(file_path):
    note_on_times = {}
    note_durations = []

    # Open the CSV file and read it
    with open(file_path, mode='r') as file:
        reader = csv.reader(file)
        
        for row in reader:
            if len(row) < 6:
                continue  # Skip any rows that don't have enough data

            # Extract the track number and check if it's between 2 and 9
            track_number = int(row[0].strip())
            if track_number < 2 or track_number > 9:
                continue  # Skip if track number is not in range 2-9
            
            # Extract the relevant values from the row
            action = row[2].strip()
            note = int(row[4])
            time = int(row[1])
            
            if action == "Note_on_c":
                # Record the time when the note was turned on, including track number
                note_on_times[(track_number, note)] = time
            elif action == "Note_off_c" and (track_number, note) in note_on_times:
                # Calculate how long the note was played
                start_time = note_on_times[(track_number, note)]
                duration = time - start_time
                note_durations.append([track_number, note, start_time, duration])
                # Remove the note from the dictionary after recording its duration
                del note_on_times[(track_number, note)]

    return note_durations


def midi_to_freq(midi):
    return(440*(2**((midi-69)/12)))

# 128 is the largest value
# 21 is the lowest




def wavelength_to_rgb(wavelength, gamma=0.8):
    """
    Converts a given wavelength of light to an approximate RGB color.
    Based on code by Dan Bruton: 
    http://www.physics.sfasu.edu/astro/color/spectra.html
    """

    if 380 <= wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma

    elif 440 < wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0

    elif 490 < wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma

    elif 510 < wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0

    elif 580 < wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0

    elif 645 < wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0

    else:
        R = 0.0
        G = 0.0
        B = 0.0

    # Scale the RGB values to 0-255
    R = int(R * 255)
    G = int(G * 255)
    B = int(B * 255)

    return([R, G, B])




In [2]:
i# Load the MIDI file and parse it into CSV format
csv_string_list = pm.midi_to_csv("/Users/abieltalwar/Downloads/Michael_Jackson_-_Billie_Jean.mid")

with open("example_converted.csv", "w") as f:
    f.writelines(csv_string_list)

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string_list)

# Save the parsed MIDI file to disk
with open("example_converted.mid", "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)

In [3]:
file_path = '/Users/abieltalwar/Documents/Python/ColouredMusic/example_converted.csv'
instrument_names = extract_instruments_from_csv(file_path)

# Output the list of instrument names
for instrument in instrument_names:
    print(instrument)

['2', 'Lead Vox']
['3', 'Elec Piano']
['4', 'Rhythm Gtr']
['5', 'Strings 2']
['6', 'Clav/Brass']
['7', 'Strings']
['8', 'Bass']
['9', 'Drums']


In [5]:
file_path = '/Users/abieltalwar/Documents/Python/ColouredMusic/example_converted.csv'
note_durations = calculate_note_durations(file_path)


[[4, 54, 79064, 1], [4, 61, 79112, 31], [4, 61, 79152, 37], [4, 50, 79207, 14], [4, 59, 79252, 48]]


In [6]:
for tple in note_durations:
    tple.append(midi_to_freq(tple[1]))
    
df = pd.DataFrame(note_durations, columns = ['Track','Note', 'Start Time','Duration', 'Music Freq' ])

m_freq_high = midi_to_freq(max(df['Note']))
m_freq_low = midi_to_freq(min(df['Note']))
l_freq_low = 400
l_freq_high = 790

def music_to_light_frequency(music):
    return(((music - m_freq_low)/(m_freq_high- m_freq_low))*(l_freq_high-l_freq_low) + l_freq_low)


df['Light Freq'] = df['Music Freq'].apply(music_to_light_frequency)

speed_of_light = 3*10**8
df['Wavelength'] = df['Light Freq'].apply(lambda freq: speed_of_light/(freq*(10**(3))))
df['RGB'] = df['Wavelength'].apply(wavelength_to_rgb)
df.to_csv("dataframe.csv")
print(df)

In [54]:
rgb_colours = df['RGB'].tolist()

# Creating plots

In [158]:
def plot_notes_for_track(df, track_number, height):

    file_name = f"track_{track_number}_notes_plot.png"
    # Filter the DataFrame for the specified track
    track_df = df[df['Track'] == track_number].reset_index(drop=True)
    
    fig, ax = plt.subplots(figsize=(10, 2))

    last_end_time = 0  # To keep track of the end time of the previous note
    
    for i, row in track_df.iterrows():
        start_time = row['Start Time']
        duration = row['Duration']
        color = row['RGB']
        norm_color = [c / 255 for c in color]  # Normalize RGB values (0-255) to (0-1)
        
        # If there is a gap between the previous note and this one, fill with black
        if start_time > last_end_time:
            ax.add_patch(plt.Rectangle((last_end_time, 0), start_time - last_end_time, height, color='black'))
        
        # Plot the note as a rectangle
        ax.add_patch(plt.Rectangle((start_time, 0), duration, height, color=norm_color))
        
        # Update the last end time
        last_end_time = start_time + duration

    # Set limits and labels
    ax.set_xlim(0, last_end_time + 100)  # Add a little padding at the end
    ax.set_ylim(0, 1)
    ax.set_aspect('auto')
    plt.axis('off')

    # Save the plot as a PNG file
    plt.savefig(file_name, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"Plot for Track {track_number} saved as '{file_name}'")


# Call the function to plot and save the notes for Track 3

for i in range(2,10):
    plot_notes_for_track(df, track_number=i,  height = 400000000)



Plot for Track 2 saved as 'track_2_notes_plot.png'
Plot for Track 3 saved as 'track_3_notes_plot.png'
Plot for Track 4 saved as 'track_4_notes_plot.png'
Plot for Track 5 saved as 'track_5_notes_plot.png'
Plot for Track 6 saved as 'track_6_notes_plot.png'
Plot for Track 7 saved as 'track_7_notes_plot.png'
Plot for Track 8 saved as 'track_8_notes_plot.png'
Plot for Track 9 saved as 'track_9_notes_plot.png'


In [163]:
def plot_notes_all_tracks(df, file_name='all_tracks_notes_plot.png', rect_height=2):
    tracks = df['Track'].unique()  # Get unique track numbers
    fig, ax = plt.subplots(figsize=(12, len(tracks) * rect_height))  # Adjust figure size based on number of tracks
    
    # Find the overall minimum start time and maximum end time across all tracks
    global_min_time = df['Start Time'].min()
    global_max_time = (df['Start Time'] + df['Duration']).max()

    # Loop through each track and plot its notes in its own row
    for track_idx, track_number in enumerate(tracks, start = 2):
        # Filter the DataFrame for the specific track
        track_idx -= 1
        print(track_idx)
        track_df = df[df['Track'] == track_number].reset_index(drop=True)

        last_end_time = global_min_time  # Set the starting time at the global minimum
        
        # Plot each note for the track
        for i, row in track_df.iterrows():
            start_time = row['Start Time']
            duration = row['Duration']
            color = row['RGB']
            norm_color = [c / 255 for c in color]  # Normalize RGB values (0-255) to (0-1)
            
            # If there is a gap between the previous note and this one, fill with black
            if start_time > last_end_time:
                ax.add_patch(plt.Rectangle((last_end_time, -track_idx * rect_height), start_time - last_end_time, rect_height, color='black'))
            
            # Plot the note as a thicker rectangle in the current track row
            ax.add_patch(plt.Rectangle((start_time, -track_idx * rect_height), duration, rect_height, color=norm_color))
            
            # Update the last end time
            last_end_time = start_time + duration

        # If there is space after the last note, fill with black
        if last_end_time < global_max_time:
            ax.add_patch(plt.Rectangle((last_end_time, -track_idx * rect_height), global_max_time - last_end_time, rect_height, color='black'))

    # Set the limits of the plot based on the common start and end times
    ax.set_xlim(global_min_time, global_max_time)
    ax.set_ylim(-len(tracks) * rect_height, 0)  # Adjust y-limits to fit all tracks
    ax.set_aspect('auto')
    plt.axis('off')

    # Save the plot as a PNG file
    plt.savefig(file_name, bbox_inches='tight', pad_inches=0)
    plt.close()

# Call the function to plot and save the notes for all tracks
plot_notes_all_tracks(df, file_name='all_tracks_notes_plot.png', rect_height=3)

print("Plot for all tracks saved as 'all_tracks_notes_plot.png'")

1
2
3
4
5
6
7
8
Plot for all tracks saved as 'all_tracks_notes_plot.png'


In [23]:
def plot_notes_all_tracks(df, file_name='all_tracks_notes_plot_with_transparency.png', rect_height=2):
    tracks = sorted(df['Track'].unique())  # Get unique track numbers, sorted
    fig, ax = plt.subplots(figsize=(12, len(tracks) * rect_height))  # Adjust figure size based on number of tracks
    
    # Find the overall minimum start time and maximum end time across all tracks
    global_min_time = df['Start Time'].min()
    global_max_time = (df['Start Time'] + df['Duration']).max()

    # Loop through each track and plot its notes in its own row
    for track_idx, track_number in enumerate(tracks, start=2):  # Track numbering starts from 2
        # Filter the DataFrame for the specific track
        track_df = df[df['Track'] == track_number].reset_index(drop=True)
        track_idx -= 1
        last_end_time = global_min_time  # Set the starting time at the global minimum
        
        # Plot each note for the track
        for i, row in track_df.iterrows():
            start_time = row['Start Time']
            duration = row['Duration']
            color = row['RGB']
            norm_color = [c / 255 for c in color]  # Normalize RGB values (0-255) to (0-1)
            
            # If there is a gap between the previous note and this one, fill with black
            if start_time > last_end_time:
                ax.add_patch(plt.Rectangle((last_end_time, -(track_idx) * rect_height), start_time - last_end_time, rect_height, color='black', alpha=1.0))
            
            # Plot the note as a rectangle with transparency
            ax.add_patch(plt.Rectangle((start_time, -(track_idx) * rect_height), duration, rect_height, color=norm_color, alpha=0.7))
            
            # Update the last end time
            last_end_time = start_time + duration

        # If there is space after the last note, fill with black
        if last_end_time < global_max_time:
            ax.add_patch(plt.Rectangle((last_end_time, -(track_idx) * rect_height), global_max_time - last_end_time, rect_height, color='black', alpha=1.0))

    # Set the limits of the plot based on the common start and end times
    ax.set_xlim(global_min_time, global_max_time)
    ax.set_ylim(-len(tracks) * rect_height, 0)  # Adjust y-limits to fit all tracks
    ax.set_aspect('auto')
    plt.axis('off')

    # Save the plot as a PNG file
    plt.savefig(file_name, bbox_inches='tight', pad_inches=0)
    plt.close()

# Call the function to plot and save the notes for all tracks with transparency
plot_notes_all_tracks(df, file_name='all_tracks_notes_plot_with_transparency.png', rect_height=3)

print("Plot for all tracks saved with transparency as 'all_tracks_notes_plot_with_transparency.png'")

Plot for all tracks saved with transparency as 'all_tracks_notes_plot_with_transparency.png'


In [29]:
# Function to plot all tracks on a single row with transparency
def plot_notes_all_tracks_single_row(df, file_name='all_tracks_single_row_plot_with_transparency.png', rect_height=2):
    tracks = sorted(df['Track'].unique())  # Get unique track numbers, sorted
    fig, ax = plt.subplots(figsize=(12, rect_height * 2))  # Adjust figure size

    # Find the overall minimum start time and maximum end time across all tracks
    global_min_time = df['Start Time'].min()
    global_max_time = (df['Start Time'] + df['Duration']).max()

    # Loop through each track and plot its notes in the same row (no vertical offset)
    for track_idx, track_number in enumerate(tracks):  # Track numbering
        # Filter the DataFrame for the specific track
        track_df = df[df['Track'] == track_number].reset_index(drop=True)

        last_end_time = global_min_time  # Set the starting time at the global minimum

        # Plot each note for the track
        for i, row in track_df.iterrows():
            start_time = row['Start Time']
            duration = row['Duration']
            color = row['RGB']
            norm_color = [c / 255 for c in color]  # Normalize RGB values (0-255) to (0-1)
            
            # If there is a gap between the previous note and this one, fill with black
            if start_time > last_end_time:
                ax.add_patch(plt.Rectangle((last_end_time, 0), start_time - last_end_time, rect_height, color='black', alpha=0.1))

            # Plot the note as a rectangle with transparency (in the same row)
            ax.add_patch(plt.Rectangle((start_time, 0), duration, rect_height, color=norm_color, alpha=0.05))

            # Update the last end time
            last_end_time = start_time + duration

        # If there is space after the last note, fill with black
        if last_end_time < global_max_time:
            ax.add_patch(plt.Rectangle((last_end_time, 0), global_max_time - last_end_time, rect_height, color='black', alpha=0.1))

    # Set the limits of the plot based on the common start and end times
    ax.set_xlim(global_min_time, global_max_time)
    ax.set_ylim(0, rect_height)  # Single row y-limits
    ax.set_aspect('auto')
    plt.axis('off')

    # Save the plot as a PNG file
    plt.savefig(file_name, bbox_inches='tight', pad_inches=0)
    plt.close()

# Call the function to plot and save the notes for all tracks on a single row
plot_notes_all_tracks_single_row(df, file_name='all_tracks_single_row_plot_with_transparency.png', rect_height=3)

print("Plot for all tracks saved with transparency as 'all_tracks_single_row_plot_with_transparency.png'")


Plot for all tracks saved with transparency as 'all_tracks_single_row_plot_with_transparency.png'


In [56]:
import matplotlib.pyplot as plt
import pandas as pd

# Function to plot all tracks on a single row with custom transparency and black background
def plot_notes_all_tracks_custom_transparency(df, alpha_list, file_name='all_tracks_black_background.png', rect_height=2):
    tracks = sorted(df['Track'].unique())  # Get unique track numbers, sorted
    fig, ax = plt.subplots(figsize=(12, rect_height * 2), facecolor='black')  # Set background color to black

    # Find the overall minimum start time and maximum end time across all tracks
    global_min_time = df['Start Time'].min()
    global_max_time = (df['Start Time'] + df['Duration']).max()

    # Loop through each track and plot its notes in the same row
    for track_idx, track_number in enumerate(tracks, start=2):  # Track numbering starts at 2
        track_idx -= 2  # Adjust for index-based logic

        # Filter the DataFrame for the specific track
        track_df = df[df['Track'] == track_number].reset_index(drop=True)

        # Get the alpha value for the current track
        alpha = alpha_list[track_idx]  # Use alpha_list for track-specific transparency

        # Plot each note for the track
        for i, row in track_df.iterrows():
            start_time = row['Start Time']
            duration = row['Duration']
            color = row['RGB']
            norm_color = [c / 255 for c in color]  # Normalize RGB values (0-255) to (0-1)

            # Plot the note as a rectangle with track-specific transparency
            ax.add_patch(plt.Rectangle((start_time, 0), duration, rect_height, color=norm_color, alpha=alpha))

    # Set the limits of the plot based on the common start and end times
    ax.set_xlim(global_min_time, global_max_time)
    ax.set_ylim(0, rect_height)  # Single row y-limits
    ax.set_aspect('auto')

    # Set the axis background to black
    ax.set_facecolor('black')
    plt.axis('off')  # Hide the axis

    # Save the plot as a PNG file
    plt.savefig(file_name, bbox_inches='tight', pad_inches=0)
    plt.close()

# Sample DataFrame for testing
# Call the function to plot and save the notes for all tracks with custom transparency and black background
alpha_list = [1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.1, 0.05]

plot_notes_all_tracks_custom_transparency(df, alpha_list, file_name='all_tracks_black_background.png', rect_height=3)

print("Plot for all tracks saved with custom transparency and black background.")


Plot for all tracks saved with custom transparency and black background.


In [43]:
import matplotlib.pyplot as plt
import pandas as pd

# Function to plot all tracks on a single row with custom transparency, filling gaps with black
def plot_notes_all_tracks_custom_transparency(df, alpha_list, file_name='all_tracks_with_gaps_filled.png', rect_height=2):
    tracks = sorted(df['Track'].unique())  # Get unique track numbers, sorted
    fig, ax = plt.subplots(figsize=(12, rect_height * 2))  # Adjust figure size

    # Find the overall minimum start time and maximum end time across all tracks
    global_min_time = df['Start Time'].min()
    global_max_time = (df['Start Time'] + df['Duration']).max()

    # Track the time intervals covered by notes
    time_covered = []

    # Loop through each track and plot its notes in the same row, allowing for overlaps
    for track_idx, track_number in enumerate(tracks):  # Track numbering without vertical separation
        # Filter the DataFrame for the specific track
        track_df = df[df['Track'] == track_number].reset_index(drop=True)
        track_idx -= 2
        # Get the alpha value for the current track (default to 0.7 if not enough values in alpha_list)
        alpha = alpha_list[track_idx]

        # Plot each note for the track and track time intervals covered
        for i, row in track_df.iterrows():
            start_time = row['Start Time']
            duration = row['Duration']
            end_time = start_time + duration
            color = row['RGB']
            norm_color = [c / 255 for c in color]  # Normalize RGB values (0-255) to (0-1)

            # Record the time covered by this note
            time_covered.append((start_time, end_time))

            # Plot the note as a rectangle with track-specific transparency (allowing overlaps)
            ax.add_patch(plt.Rectangle((start_time, 0), duration, rect_height, color=norm_color, alpha=alpha))

    # Sort time intervals by start time and merge overlapping ones
    time_covered = sorted(time_covered)
    merged_intervals = []
    current_start, current_end = time_covered[0]

    for start, end in time_covered[1:]:
        if start <= current_end:  # Overlapping or adjacent intervals
            current_end = max(current_end, end)
        else:
            merged_intervals.append((current_start, current_end))
            current_start, current_end = start, end
    merged_intervals.append((current_start, current_end))  # Add the last interval

    # Now fill in the gaps with black between merged time intervals
    last_end_time = global_min_time
    for start, end in merged_intervals:
        if last_end_time < start:
            ax.add_patch(plt.Rectangle((last_end_time, 0), start - last_end_time, rect_height, color='black', alpha=1.0))
        last_en3_time = end

    # Fill any gap at the end of the timeline
    if last_end_time < global_max_time:
        ax.add_patch(plt.Rectangle((last_end_time, 0), global_max_time - last_end_time, rect_height, color='black', alpha=1.0))

    # Set the limits of the plot based on the common start and end times
    ax.set_xlim(global_min_time, global_max_time)
    ax.set_ylim(0, rect_height)  # Single row y-limits
    ax.set_aspect('auto')
    plt.axis('off')  # Hide the axis

    # Save the plot as a PNG file
    plt.savefig(file_name, bbox_inches='tight', pad_inches=0)
    plt.close()

# Sample DataFrame for testing
# Alpha values corresponding to tracks (track 2 is more transparent than track 3)
alpha_list = [1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# Call the function to plot and save the notes for all tracks with custom transparency and gaps filled with black
plot_notes_all_tracks_custom_transparency(df, alpha_list, file_name='all_tracks_with_gaps_filled.png', rect_height=3)

print("Plot for all tracks saved with custom transparency and gaps filled with black.")


Plot for all tracks saved with custom transparency and gaps filled with black.
